# Introduction

Initial Particle Positions:

Particles are initialized at the intersection of Lake Huron watersheds with the shoreline.
To ensure starting points are within the lake, these intersection points are snapped 50 meters toward the lake.
The create initial position module is used to generate multiple particles around each primary starting point for enhanced resolution.

# Creating the initial position file using watershed pourpoints
The following code read the pour points extracted from Arc gis from the intersection of watersheds with lake huron boundary.
 We are using PyLag Create initial position function to Create a release Zone for each of the Pour points. Before creating the initial positions, We want to add a condition to make a the relase zone that are in the boundary of Lake Huron.
 
 Then using Pylag create initial position file it will create a release zone for each pour point/groupid, in the end each group_id will contain sets of release zone 
 6/3/2024

#### Part 1-1: Import required libraries

In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
import matplotlib.pyplot as plt
from pylag.processing.coordinate import get_epsg_code, utm_from_lonlat, lonlat_from_utm
from pylag.processing.coordinate import utm_from_lonlat, lonlat_from_utm
from pylag.processing.release_zone import create_release_zones_along_cord
from pylag.processing.plot import create_figure, colourmap
from pylag.processing.plot import FVCOMPlotter
from pylag.processing.release_zone import create_release_zone
from pylag.processing.input import create_initial_positions_file_multi_group
from pylag.processing.input import create_initial_positions_file_single_group

# Input path/ files

In [4]:
# path to pourpoint 
pourpoints = r"/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/pourpoints"

#init_file = os.path.join(pourpoints, 'WatershedPourPoints_NAD1983.dat')

init_file = os.path.join(pourpoints, 'Streamwatershedpourpoints_snapped_middleplume.shp')


# Output path

In [4]:
pourpoint_dat = os.path.join(pourpoints, 'Streamwatershedpourpoints_NAD1983_buffer_middleplume.dat')

# path to the release zone that intersect to Lake Huron
# path to release zone shapefile
release_zone_path = r"/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/particle_tracking_output"
intersection = "initial_positions_releasezone_intersection_middleplume.shp"
release_zone_intersect = os.path.join(release_zone_path, 'initial_positions_releasezone_intersection_middleplume.shp')

# temporary file to store the release zone

temp_out_name = os.path.join(release_zone_path, 'initial_positions_releasezone_middleplume.shp')

In [5]:
# path to final output file
out_path = '/home/abolmaal/modelling/FVCOM/Huron/input/initial_position'

# name of the output file that is multigrouped and ready to be used in FVCOM
output_file_initial_releasezone_multigroup = os.path.join(out_path, 'initial_positions_releasezone_intersection_multigroup_middleplume.dat')

output_file_initial_releasezone_multigroup_lastrevised= os.path.join(out_path, 'initial_positions_releasezone_intersection_multigroup_midplume_lastrevised.dat')


# Evironments
create directory/Folders

#### Part1-2: Read the initial position file(from watreshed and lake intersections) and rename the columns

# Parameters/ temporary

In [60]:
# specify the parameters to Create release zone
# Release zone  
radius = 0.006
# target number of particles
n_particles_target = 100

# Release depths
depth_below_surface = 0.0

# define the watershed number
watershed_num = "group_id"

# Reading the data

In [6]:
# Converting the shapefile to a dat file and keeping the required columns
# this is the output from arcgis and in this code we are preparing the output to be used for particle tracking

In [61]:
# read the pourpoint shapefile
df_pourpoint = gpd.read_file(init_file)
# print the first 5 rows
# rename the columns POINT_X and POINT_Y to lat and lon
df_pourpoint_copy = df_pourpoint.rename(columns={'POINT_X': 'lon', 'POINT_Y': 'lat'})
# add a new column called deoth and set it to 0
df_pourpoint_copy['depth'] = 0

# conver the crs to lonlat
df_pourpoint_copy = df_pourpoint_copy.to_crs(epsg=4326)

# conver columns lon and lat to epsg 4326
df_pourpoint_copy['lon'] = df_pourpoint_copy['geometry'].x
df_pourpoint_copy['lat'] = df_pourpoint_copy['geometry'].y

# if column lon is not in the range of -180 to 0, then convert it to the range of 0 to 360
df_pourpoint_copy.loc[df_pourpoint_copy['lon'] < 0, 'lon'] = df_pourpoint_copy.loc[df_pourpoint_copy['lon'] < 0, 'lon'] + 360
df_pourpoint_copy = df_pourpoint_copy[['Group_id', 'lon', 'lat', 'depth']]


print(df_pourpoint_copy.head())

# save the lon and lat with only four decimal points
df_pourpoint_copy['lon'] = df_pourpoint_copy['lon'].round(4)
df_pourpoint_copy['lat'] = df_pourpoint_copy['lat'].round(4)

# save the pourpoint file to a dat file with one space as a separator
df_pourpoint_copy.to_csv(pourpoint_dat, sep=' ', index=False, header=False)

   Group_id         lon        lat  depth
0         0  275.336267  45.747577      0
1         3  275.882160  45.535793      0
2         4  276.076128  45.494006      0
3         9  276.176836  45.432871      0
4        10  276.285434  45.418564      0


In [62]:
# reading the pouropoints file

df_pourpoint = pd.read_csv(pourpoint_dat, skiprows=0,sep=' ',header=None)
# rename the columns
df_pourpoint.rename(columns={0: "Group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)
df_pourpoint

,Group_id,lon,lat,depth
0,0,275.3363,45.7476,0
1,3,275.8822,45.5358,0
2,4,276.0761,45.4940,0
3,9,276.1768,45.4329,0
4,10,276.2854,45.4186,0
...,...,...,...,...
109,144,277.5285,43.1226,0
110,145,277.5489,43.0819,0
111,146,277.5618,43.0711,0
112,147,277.5855,43.0251,0


# Main Function 

# Create a release zone and save it

In [63]:
# list to accumulate release zones
release_zones = []

# Loop through the rows of the DataFrame
for index, row in df_pourpoint.iterrows():
    group_id = row['Group_id']
    lat = row['lat']
    lon = row['lon']
    
    # Create the release zone
    surface_release_zone = create_release_zone(group_id = group_id,
                                           radius = radius,
                                           centre = [lon, lat],
                                           n_particles = n_particles_target,
                                           depth = depth_below_surface,
                                           random = False)
    
    # accumulate the release zones
    release_zones.append(surface_release_zone)
    

# Write data to file
create_initial_positions_file_multi_group(output_file_initial_releasezone_multigroup, release_zones)


In [64]:
output_file_initial_releasezone_multigroup

'/home/abolmaal/modelling/FVCOM/Huron/input/initial_position/initial_positions_releasezone_intersection_multigroup_middleplume.dat'

In this part I save the initial position file as shape file and I used arcpy to remove the points that are not inside Lake Huron. 

Code for removing the points is in InDirectNutrientLoad_Arcpy directory and it's called Initial_position_removeoutsidelakepoints

In [65]:
initial_positions = output_file_initial_releasezone_multigroup

In [66]:
# Read the output file
df_initial_positions = pd.read_csv(initial_positions, sep=' ', header=None, skiprows=1)


In [67]:
# Rename the columns
df_initial_positions.rename(columns={0: "Group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)

# round the lon and lat to 4 decimal points
df_initial_positions['lon'] = df_initial_positions['lon'].round(4)
df_initial_positions['lat'] = df_initial_positions['lat'].round(4)

# group_id to integer
df_initial_positions['Group_id'] = df_initial_positions['Group_id'].astype(int)
# depth integer
df_initial_positions['depth'] = df_initial_positions['depth'].astype(int)
# Calculate the total number of particles
n_particles_target = df_initial_positions['Group_id'].count()

# Display `n_particles_target` as part of the header information
print(f"# Total particles: {n_particles_target}")
df_initial_positions.head()

# Total particles: 8550


,Group_id,lon,lat,depth
0,0,275.3339,45.7428,0
1,0,275.3351,45.7428,0
2,0,275.3363,45.7428,0
3,0,275.3375,45.7428,0
4,0,275.3387,45.7428,0


In [68]:
# Save the output file
# Write only the total number of particles as the header and data without column names
with open(output_file_initial_releasezone_multigroup_lastrevised, 'w') as file:
    # Write the total number of particles as the header
    file.write(f"{n_particles_target}\n")
    # Write the data without headers
    df_initial_positions.to_csv(file,sep=' ' ,index=False, header=False)

In [69]:
# convert the out_file_initial_releasezone_multigroup_lastrevised to a shapefile
# read the output file
df_initial_positions = pd.read_csv(output_file_initial_releasezone_multigroup_lastrevised, sep=' ', header=None, skiprows=1)
# Rename the columns
df_initial_positions.rename(columns={0: "Group_id", 1:"lon",2:"lat",3:"depth"}, inplace=True)
# round the lon and lat to 4 decimal points
df_initial_positions['lon'] = df_initial_positions['lon'].round(4)
df_initial_positions['lat'] = df_initial_positions['lat'].round(4)
# group_id to integer
df_initial_positions['Group_id'] = df_initial_positions['Group_id'].astype(int)


In [70]:
df_temp = df_initial_positions.copy()

In [71]:
df_temp['lon'] = df_temp['lon'] -360

In [72]:
df_temp


,Group_id,lon,lat,depth
0,0,-84.6661,45.7428,0
1,0,-84.6649,45.7428,0
2,0,-84.6637,45.7428,0
3,0,-84.6625,45.7428,0
4,0,-84.6613,45.7428,0
...,...,...,...,...
8545,162,-83.8535,43.6591,0
8546,162,-83.8523,43.6591,0
8547,162,-83.8511,43.6591,0
8548,162,-83.8499,43.6591,0


# note define this top

In [73]:
# save the dataframe as a shapefile in temp_out directory with temp_out_name
gdf = gpd.GeoDataFrame(df_temp, geometry=gpd.points_from_xy(df_temp.lon, df_temp.lat))
# this is written as WGS84
gdf.crs = 'EPSG:4326'
gdf.to_file(temp_out_name)

In [74]:
temp_out_name

'/mnt/d/Users/abolmaal/Arcgis/NASAOceanProject/GIS_layer/particle_tracking_output/initial_positions_releasezone_middleplume.shp'

# 3-Solve an error related to PyLag

My Lat and lon number has more than 10 digits and when I run it through Pylag it gives me this error:ValueError: invalid literal for int() with base 10: '0.0' so to avoid this I want to read Lat and Lon and round them by 10


Once I removed the points outside Lake Boundary I read the cleaned point here again 
- make sure Longitudes are in 360 degree 
- Lat and Lon are rounded to 10 
- Save the file as dat file



In [23]:
# read the release zone shapefile that intersect with Lake Huron
release_zone_intersect = gpd.read_file(release_zone_intersect)
release_zone_intersect.head()

,FID_initia,group_id,lon,lat,depth,FID_hydro_,HYDRO_P_,UIDENT,TYPE,COUNTRY,NAMEEN,NAMESP,NAMEFR,geometry
0,18205,124.0,277.574766,43.019242,0.0,0,3261,552202,16,USA,Lake Huron,Lake Huron/Lac Huron,Lac Huron,POINT (-82.42523 43.01924)
1,18216,124.0,277.573338,43.020670,0.0,0,3261,552202,16,USA,Lake Huron,Lake Huron/Lac Huron,Lac Huron,POINT (-82.42666 43.02067)
2,18217,124.0,277.574766,43.020670,0.0,0,3261,552202,16,USA,Lake Huron,Lake Huron/Lac Huron,Lac Huron,POINT (-82.42523 43.02067)
3,18218,124.0,277.576195,43.020670,0.0,0,3261,552202,16,USA,Lake Huron,Lake Huron/Lac Huron,Lac Huron,POINT (-82.42381 43.02067)
4,18228,124.0,277.571909,43.022099,0.0,0,3261,552202,16,USA,Lake Huron,Lake Huron/Lac Huron,Lac Huron,POINT (-82.42809 43.02210)


In [ ]:
# keep only the group_id, lon, lat, and depth columns
release_zone_intersect = release_zone_intersect[['group_id', 'lon', 'lat', 'depth']]
# sort the data from group_id
release_zone_intersect = release_zone_intersect.sort_values(by=[watershed_num])
# if df[lon] and df[lat] are greater than 10 digits round them to 10 digits
release_zone_intersect['lon'] = release_zone_intersect['lon'].round(10)
release_zone_intersect['lat'] = release_zone_intersect['lat'].round(10)
# group_id to integer
release_zone_intersect['group_id'] = release_zone_intersect['group_id'].astype(int)
# depth integer
release_zone_intersect['depth'] = release_zone_intersect['depth'].astype(int)
# Calculate the total number of particles
n_particles_target = release_zone_intersect['group_id'].count()

# Display `n_particles_target` as part of the header information
print(f"# Total particles: {n_particles_target}")
release_zone_intersect.head()

# Total particles: 8889


,group_id,lon,lat,depth
8888,0,275.334153,45.755069,0
8831,0,275.338439,45.746498,0
8832,0,275.339868,45.746498,0
8833,0,275.341296,45.746498,0
8834,0,275.342725,45.746498,0


# Save outputs

In [25]:
# Write only the total number of particles as the header and data without column names
with open(output_file_initial_releasezone_multigroup, 'w') as file:
    # Write the total number of particles as the header
    file.write(f"{n_particles_target}\n")
    # Write the data without headers
    release_zone_intersect.to_csv(file,sep=' ' ,index=False, header=False)